In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
# load data

data = pd.read_csv("Churn_Modelling.csv")

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.shape

(10000, 14)

Data pre-processing

In [25]:
# drop irrelevant variables

data_v1 = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [26]:
# binary encoding to gender variable
label_encoder = LabelEncoder()
data_v1['Gender'] = label_encoder.fit_transform(data_v1['Gender'])

In [27]:
data_v1['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [28]:
# onehotencoding for geography variable

one_hot_encoder = OneHotEncoder(sparse_output=False)
geo_encoder = one_hot_encoder.fit_transform(data[['Geography']])
print(geo_encoder)
print(one_hot_encoder.get_feature_names_out())
# create dataframe

geo_encoder_df = pd.DataFrame(geo_encoder,columns=one_hot_encoder.get_feature_names_out())
display(geo_encoder_df.head())
# join both dataframes

data_v2 = pd.concat([data_v1.drop('Geography',axis=1),geo_encoder_df],axis=1)
data_v2.head()

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
['Geography_France' 'Geography_Germany' 'Geography_Spain']


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [29]:
# save the label encoder and onehot encoder object into pickle file
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder,file)


with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder,file)


Train-Test split

In [30]:
# Divide data into independent and dependent variables

X = data_v2.drop('Exited',axis=1) # independent variables
y = data_v2['Exited'] # dependent variables / target variables

# split the data into train and test datasets
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42,shuffle=True)

In [33]:
# scale the features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
# save the scaler object into pickle file
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [39]:
model = Sequential([
    Input(shape=(12,)), #input layer
    Dense(64,activation='relu'), # hidden layer 1
    Dense(32,activation='relu'), # hidden layer 2
    Dense(1,activation='sigmoid') # output layer

]
)

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


- Many practitioners start with a larger number of neurons in the first layer, gradually decreasing in subsequent layers. This is because the first layer needs to capture a wide range of features and patterns.
- The number of neurons often correlates with the complexity of the problem. If your dataset is complex, you might need more neurons to capture the nuances.
- Using too many neurons can lead to overfitting, especially with smaller datasets. Reducing the number of neurons in subsequent layers can help mitigate this.
- A common approach is to use a "pyramid" design, where the first layer has more neurons and each subsequent layer has fewer. This structure can help the model generalize better.

In [41]:
# compile the model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [42]:
# set up tensorboard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

# set up early stopping
early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [43]:
# train the model
history = model.fit(
    X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=100,batch_size=32,
    callbacks=[tensorflow_callback,early_stopping]
)

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 0.4040 - accuracy: 0.8303 - val_loss: 0.3614 - val_accuracy: 0.8445
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3537 - accuracy: 0.8584 - val_loss: 0.3532 - val_accuracy: 0.8640
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3488 - accuracy: 0.8572 - val_loss: 0.3423 - val_accuracy: 0.8640
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3436 - accuracy: 0.8591 - val_loss: 0.3542 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3411 - accuracy: 0.8593 - val_loss: 0.3531 - val_accuracy: 0.8460
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3372 - accuracy: 0.8595 - val_loss: 0.3377 - val_accuracy: 0.8580
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3362 - accuracy: 0.8629 - val_loss: 0.3434 - val_accuracy: 0.85

In [ ]:
# save the model
model.save('model.h5')

In [44]:
# lauch tensorboard
%load_ext tensorboard

In [45]:
%tensorboard --logdir logs/fit20241017-234534